Carregando os dados necessários

In [3]:
from pandas import read_csv, to_datetime
from os import path

datadir = '/home/iagonmic/data_science/ecommerce-joias/_data/clean'

df_clientes = read_csv(path.join(datadir, 'e_commerce_clientes.csv'), decimal=',')
df_carrinhos = read_csv(path.join(datadir, 'e_commerce_carrinhos.csv'), decimal=',')
df_pedidos = read_csv(path.join(datadir, 'e_commerce_pedidos.csv'), decimal=',')

df_pedidos.drop(columns=['Unnamed: 0']).reset_index(drop=True).head()

,id,data,data_pagamento,data_cancelamento,numero_pedido,status,pagamento,produto,sku,quantidade,...,entrega_cidade,entrega_estado,entrega_cep,utm_source,utm_campaign,utm_medium,utm_content,utm_term,customizacao,id_cliente
0,117569438,04/08/2024 18:26:40,04/08/2024 18:27,NaN,1042552530523300,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,1,...,Brasilia,Distrito Federal,72455490,NaN,NaN,NaN,NaN,NaN,NaN,31d69aed10
1,117385907,01/08/2024 19:08:02,01/08/2024 19:11,NaN,1042552893870166,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,1,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
2,117385794,01/08/2024 19:05:53,NaN,01/08/2024 19:12,1042552190094688,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,1,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
3,117382873,01/08/2024 18:12:14,NaN,01/08/2024 19:13,1042552273445602,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,1,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
4,117382771,01/08/2024 18:09:41,NaN,01/08/2024 19:13,1042552782939597,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,1,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5


OBS: ao verificarmos quantos clientes ativos temos através da colunas ativos usando df_clientes.query("ativo == 'sim'"), temos uma quantidade incorreta de clientes ativos, mostrando 83 de 84, sendo que, ao verificarmos a quantidade de clientes únicos através do dataset de pedidos temos 43 clientes que fizeram pedidos.

Realizando os tratamentos dos dados necessários em df_pedidos:

Do dataframe 'df_pedidos' Iremos pegar a coluna 'total_pago' e a coluna 'id_cliente', assim teremos o valor total pago por cliente, além da data. A ideia é dividir em dois grupos, o ticket médio dos clientes em datas normais, e em datas especiais.

In [10]:
# Trocando as ',' por '.' para converter para float na coluna 'total_pago'
df_pedidos['total_pago'] = (df_pedidos['total_pago']
 .replace(',', '.', regex=True)
 .astype({'total_pago': float}))

# Trocando o tipo da coluna 'data' para datetime (formato do pandas para datas)
df_pedidos['data'] = to_datetime(df_pedidos['data'], format='%d/%m/%Y %H:%M:%S')
df_pedidos['data'] = df_pedidos['data'].apply(lambda x: x.strftime('%Y/%m/%d'))

# Filtrando apenas as colunas necessárias do dataframe 'df_pedidos'
df_pedidos = df_pedidos.filter(['id_cliente', 'total_pago', 'data'])

df_pedidos

,id_cliente,total_pago,data
0,31d69aed10,508.90,2024/08/04
1,3e109eedd5,518.90,2024/08/01
2,3e109eedd5,518.90,2024/08/01
3,3e109eedd5,518.90,2024/08/01
4,3e109eedd5,518.90,2024/08/01
...,...,...,...
140,0db3cf4f27,14.53,2024/05/22
141,0db3cf4f27,13.50,2024/05/22
142,0db3cf4f27,139.30,2024/05/22
143,0db3cf4f27,34.90,2024/05/22


 Criando df_pedidos com datas especiais e não especiais

In [11]:
# Vamos criar a lista com as datas especiais
datas_especiais = ([
    '2024/01/01', '2024/01/06', '2024/01/25', '2024/02/10', '2024/02/11', 
    '2024/02/12', '2024/02/13', '2024/02/14', '2024/02/24', '2024/03/08', 
    '2024/03/10', '2024/03/19', '2024/03/29', '2024/04/01', '2024/04/21', 
    '2024/04/22', '2024/04/25', '2024/04/30', '2024/05/01', '2024/05/13', 
    '2024/05/31', '2024/06/05', '2024/06/12', '2024/06/24', '2024/07/02', 
    '2024/07/09', '2024/07/20', '2024/08/11', '2024/08/15', '2024/08/19', 
    '2024/09/07', '2024/09/29', '2024/10/12', '2024/10/15', '2024/10/28', 
    '2024/11/02', '2024/11/15', '2024/11/20', '2024/12/24', '2024/12/25', '2024/12/31'])

# Agora criaremos outro dataframe apenas com as datas_especiais e um com datas normais
df_pedidos_datas_especiais = df_pedidos[df_pedidos['data'].isin(datas_especiais)].reset_index(drop=True)
df_pedidos_datas_normais = df_pedidos[df_pedidos['data'].isin(datas_especiais) == False].reset_index(drop=True)

In [12]:
df_pedidos_datas_especiais

,id_cliente,total_pago,data
0,69ae2c2675,142.90,2024/07/09
1,69ae2c2675,291.90,2024/07/09
2,69ae2c2675,291.90,2024/07/09
3,69ae2c2675,637.90,2024/07/09
4,69ae2c2675,637.90,2024/07/09
5,f3d6007fb0,464.90,2024/07/09
6,f3d6007fb0,464.90,2024/07/09
7,bd425d1832,577.00,2024/06/12
8,bd425d1832,577.00,2024/06/12
9,bd425d1832,577.00,2024/06/12


In [13]:
df_pedidos_datas_normais

,id_cliente,total_pago,data
0,31d69aed10,508.90,2024/08/04
1,3e109eedd5,518.90,2024/08/01
2,3e109eedd5,518.90,2024/08/01
3,3e109eedd5,518.90,2024/08/01
4,3e109eedd5,518.90,2024/08/01
...,...,...,...
124,0db3cf4f27,14.53,2024/05/22
125,0db3cf4f27,13.50,2024/05/22
126,0db3cf4f27,139.30,2024/05/22
127,0db3cf4f27,34.90,2024/05/22


In [14]:
# Vamos criar os grupos com o ticket médio de cada cliente
ticket_medio_especiais = df_pedidos_datas_especiais.groupby('id_cliente')['total_pago'].mean()
ticket_medio_normais = df_pedidos_datas_normais.groupby('id_cliente')['total_pago'].mean()

In [15]:
ticket_medio_especiais # Limitação com apenas 5 linhas

id_cliente
69ae2c2675    400.50
6d89f121b2    607.00
af067145fb    619.09
bd425d1832    577.00
f3d6007fb0    464.90
Name: total_pago, dtype: float64

In [16]:
ticket_medio_normais

id_cliente
0220acd6a2    166.900000
0db3cf4f27     70.288000
109b4796fb    525.890000
1c21b7936a    185.900000
20b59787f0    318.470000
25a073ec7e    121.440000
31d69aed10    508.900000
3945c23d9d    386.000000
3c4f7f5500    506.850000
3d733877be    234.960000
3e109eedd5    518.900000
44084680ed    186.820000
45edbd16cf    365.325714
49963620ad    153.020000
4f62898592    168.900000
5750bce6a8    290.210000
58cce37349    379.840000
670d1fc5e4    136.500000
68a244dc63    259.460000
75a57b5380    238.290000
7b4fa02eda    339.035000
82b202fdb5    461.250000
8822bf59c1    322.040000
952b97a1ec    386.951429
a01aae65a5    490.269333
a57565d68a    362.802500
af067145fb    393.000000
c3de12c313    507.680000
c9233b915e    362.310000
ce08f9bda2    168.900000
d1aacacb7f    296.300000
d9c52cd22c    393.000000
dc1ba1366f    159.510000
dcef5308d0    870.313077
ddef8dfacc    418.900000
dee4105753    131.660000
ebd181dfc4    182.660000
f026eb1e16    200.770000
f3d6007fb0    374.487500
fe8e6062bc    

No código acima, percebemos que df_pedidos_datas_especiais temos uma limitação, pois temos uma amostra relativamente pequena, contendo apenas 5 clientes de 43 enquanto que no de datas normais temos 40 linhas, ou seja, uma diferença de 8x no tamanho da amostra.

Agora, como queremos comparar a variável 'ticket médio' entre dois grupos, e ela é contínua. Com isso, iremos usar o teste t two sample se for normalmente distribuida (paramétrico), ou o teste de mann whitney u se não for paramétrico.

In [23]:
from scipy.stats import shapiro

res_especiais = shapiro(ticket_medio_especiais)
res_normais = shapiro(ticket_medio_normais)

print(f"O p-valor para datas especiais é {res_especiais.pvalue}")
print(f"O p-valor para datas normais é {res_normais.pvalue}")

O p-valor para datas especiais é 0.271522730038582
O p-valor para datas normais é 0.006793495089499543


Nesse caso, como os p-valores estão dando resultados diferentes, sendo um para teste paramétrico e outro para não paramétrico, iremos considerar o que tem o maior tamanho amostral, ou seja, o p-valor de datas normais, que é 8x maior que o de datas especiais. Portanto, o p-valor é 0,6%, o que é menor que 5%, resultando em um teste não paramétrico. Vamos usar o Wilcoxon Rank Sum (ou Mann Whitney U) com a hipótese H1 alternativa de que o valor do ticket medio em datas especiais é maior.

In [21]:
from scipy.stats import mannwhitneyu

_, p_valor = mannwhitneyu(ticket_medio_especiais, ticket_medio_normais, alternative='greater')

print(f"O p-valor do teste de hipótese é: {p_valor}")
print(f"A mediana do ticket médio em datas normais é {ticket_medio_normais.median()}")
print(f"A mediana do ticket médio em datas especiais é {ticket_medio_especiais.median()}")

O p_valor do teste de hipótese é: 0.001821594097401349
A mediana do ticket médio em datas normais é 320.255
A mediana do ticket médio em datas especiais é 577.0


Como a hipótese H0 nula foi rejeitada devido ao p-valor menor que 5%, temos que a hipótese alternativa de que o ticket médio em datas especiais é maior do que o ticket médio em datas normais é o resultado do teste de Mann Whitney U. Além disso, podemos verificar pela diferença das medianas também, onde a mediana em datas normais (320,26) é menor do que a de datas especiais (577), o que mostra que o ticket médio é maior em datas especiais.